In [1]:
# Setup the database and save the data
import duckdb
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)


with duckdb.connect(database='../data/database.duckdb', read_only=False) as con:
    # Use this to open and close the connection, so that the database is not locked
    pass
    

In [2]:
# First we load the data into a pandas dataframe, just what we are used to.
# We can do the same just with Duckdb though but it shows nicely how well Duckdb works with Pandas.
df = pd.read_csv(
    '../data/ground-truth/data.csv', 
    sep=";", 
    decimal=',', 
    low_memory=False,
    parse_dates=['LastPaymentDate', 'PenultimatePaymentDate', 'Date of Birth'], 
    dayfirst=True
)
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.dtypes


customer_number                    int64
gender                            object
date_of_birth             datetime64[ns]
postcode                          object
count2015                          int64
sum2015                          float64
merchandise2015                    int64
count2016                          int64
sum2016                          float64
merchandise2016                    int64
count2017                          int64
sum2017                          float64
merchandiese2017                   int64
count2018                          int64
sum2018                          float64
merchandiese2018                   int64
count2019                          int64
sum2019                          float64
merchandise2019                    int64
lastpaymentdate           datetime64[ns]
penultimatepaymentdate    datetime64[ns]
dtype: object

In [3]:

# Customers with added age column
with duckdb.connect(database='../data/database.duckdb', read_only=False) as con:
    con.execute("""
        CREATE OR REPLACE TABLE customers AS (
            SELECT 
                *,
                STRING_SPLIT(
                    CAST((CURRENT_DATE() - date_of_birth) / 365 AS STRING), 
                    ' '
                )[1]::INTEGER as age, 
            FROM df)
    """)
    con.execute("""
        SELECT * FROM customers LIMIT 1
    """)
    print(con.fetchall())


[(1, 'female', None, None, 0, 0.0, 0, 0, 0.0, 0, 0, 0.0, 0, 0, 0.0, 0, 1, 4.5, 0, datetime.datetime(2019, 12, 18, 0, 0), None, None)]


In [11]:
# In hindsight, this is not the best way to do this. Just create the whole table and let superset slice and dice the data.
create_age_distribution = """
CREATE OR REPLACE TABLE age_distribution AS (
  WITH age AS (
      SELECT
          STRING_SPLIT(
              CAST((CURRENT_DATE() - date_of_birth) / 365 AS STRING), 
              ' '
          )[1]::INTEGER as age,
      FROM df
      WHERE age is not NULL
  )
  SELECT
      age,
      count(*) AS count,
  FROM age
  GROUP BY age
  ORDER BY age asc
)
"""

with duckdb.connect(database='../data/database.duckdb', read_only=False) as con:
    con.execute(create_age_distribution)
    con.execute("""
        SELECT * FROM age_distribution LIMIT 1
    """)
    print(con.fetchall())

[(7, 1)]


In [4]:
create_rfc_quantiles = """
CREATE OR REPLACE TABLE rfc_quantiles AS (
    WITH summary_data AS (
        SELECT
            customer_number,
            sum(sum2015 + sum2016 + sum2017 + sum2018 + sum2019) AS total_sum,
            sum(count2015 + count2016 + count2017 + count2018 + count2019) AS total_count,
            (SELECT MAX(lastpaymentdate) from df) - lastpaymentdate AS days_since_last_payment,
        FROM df
        GROUP BY customer_number, lastpaymentdate
    ),
    quantiles AS (
        SELECT
            customer_number,
            quantile(total_sum, 0.25) AS total_sum_1,
            quantile(total_sum, 0.5) AS total_sum_2,
            quantile(total_sum, 0.75) AS total_sum_3,
            quantile(total_count, 0.25) AS total_count_1,
            quantile(total_count, 0.5) AS total_count_2,
            quantile(total_count, 0.75) AS total_count_3,
            quantile(days_since_last_payment, 0.25) AS days_since_last_payment_1,
            quantile(days_since_last_payment, 0.5) AS days_since_last_payment_2,
            quantile(days_since_last_payment, 0.75) AS days_since_last_payment_3,
        FROM summary_data
        GROUP BY customer_number
    ),
    quantile_binned AS (
        SELECT
            quantiles.customer_number,
            CASE
                WHEN total_sum < total_sum_1 THEN 1
                WHEN total_sum < total_sum_2 THEN 2
                WHEN total_sum < total_sum_3 THEN 3
                ELSE 4
            END AS monetary_quantile,
            CASE
                WHEN total_count < total_count_1 THEN 1
                WHEN total_count < total_count_2 THEN 2
                WHEN total_count < total_count_3 THEN 3
                ELSE 4
            END AS frequency_quantile,
            -- Reversed the order of the quantiles for recency
            CASE
                WHEN days_since_last_payment < days_since_last_payment_1 THEN 4
                WHEN days_since_last_payment < days_since_last_payment_2 THEN 3
                WHEN days_since_last_payment < days_since_last_payment_3 THEN 2
                ELSE 1
            END AS recency_quantile,
        FROM quantiles 
        JOIN summary_data ON (quantiles.customer_number = summary_data.customer_number)
    )
    SELECT
        CASE
            WHEN monetary_quantile >= 4 AND frequency_quantile >= 4 AND recency_quantile >= 4 THEN 'Champions'
            WHEN monetary_quantile >= 4 AND frequency_quantile >= 4 AND recency_quantile <= 1 THEN 'Lost Champions'
            WHEN monetary_quantile >= 3 AND frequency_quantile >= 3 AND recency_quantile >= 3 THEN 'Loyal Customers'
            WHEN monetary_quantile >= 3 AND frequency_quantile >= 2 AND recency_quantile >= 4 THEN 'Potential Loyalists'
            WHEN monetary_quantile >= 2 AND frequency_quantile >= 1 AND recency_quantile >= 4 THEN 'New Customers'
            WHEN monetary_quantile >= 2 AND frequency_quantile >= 1 AND recency_quantile >= 3 THEN 'Promising'
            WHEN monetary_quantile >= 2 AND frequency_quantile >= 1 AND recency_quantile >= 2 THEN 'Hibernating'
            WHEN monetary_quantile >= 1 AND frequency_quantile >= 1 AND recency_quantile >= 1 THEN 'Lost'
        END AS customer_segment,
        STRING_SPLIT(
            CAST((CURRENT_DATE() - date_of_birth) / 365 AS STRING), 
            ' '
        )[1]::INTEGER as age,
        *
    FROM quantile_binned
    LEFT JOIN df USING (customer_number)
    LEFT JOIN summary_data USING (customer_number)
)
"""

with duckdb.connect(database='../data/database.duckdb', read_only=False) as con:
    con.execute(create_rfc_quantiles)
    con.execute("""
        SELECT * FROM rfc_quantiles LIMIT 1
    """)
    print(con.fetchall())

[('Lost Champions', None, 307201, 4, 4, 1, 'female', None, '1210', 2, 7.5, 0, 1, 10.0, 0, 0, 0.0, 0, 2, 10.5, 0, 0, 0.0, 0, datetime.datetime(2018, 11, 9, 0, 0), datetime.datetime(2018, 11, 8, 0, 0), 28.0, 5, datetime.timedelta(days=461))]
